In [1]:
import pandas as pd
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
import torch

c:\Users\iohkg\miniconda3\envs\dl\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cpu


In [70]:
# Load dataset
dataset = load_dataset('csv', data_files='large_vocab_refined.txt')

Generating train split: 0 examples [00:00, ? examples/s]c:\Users\iohkg\miniconda3\envs\dl\lib\site-packages\datasets\download\streaming_download_manager.py:778: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating train split: 8563 examples [00:00, 42981.31 examples/s]


In [71]:
dataset

DatasetDict({
    train: Dataset({
        features: ['domain', 'general'],
        num_rows: 8563
    })
})

In [72]:
# Load tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
# Load model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

In [73]:
def preprocess_data(data):
    inputs = data['general']
    targets = data['domain']
    model_inputs = tokenizer(inputs, max_length=10, truncation=True, padding='max_length')
    
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=4, truncation=True,  padding='max_length')
    
    labels["input_ids"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in labels_example] 
        for labels_example in labels["input_ids"]
    ]

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [74]:
tokenized_dataset = dataset.map(preprocess_data, batched=True,  remove_columns=["general", "domain"])

Map:   0%|          | 0/8563 [00:00<?, ? examples/s]c:\Users\iohkg\miniconda3\envs\dl\lib\site-packages\transformers\tokenization_utils_base.py:4144: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 8563/8563 [00:00<00:00, 9168.95 examples/s] 


In [75]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8563
    })
})

In [76]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['train'],
)

c:\Users\iohkg\miniconda3\envs\dl\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [77]:
# Fine-tune the model
trainer.train()

  4%|▍         | 500/12846 [20:14<8:12:35,  2.39s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'loss': 2.4679, 'grad_norm': 55.49236297607422, 'learning_rate': 1.9221547563443874e-05, 'epoch': 0.12}


  8%|▊         | 1000/12846 [40:19<8:00:02,  2.43s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'loss': 1.995, 'grad_norm': 44.26969528198242, 'learning_rate': 1.844309512688775e-05, 'epoch': 0.23}


 12%|█▏        | 1500/12846 [59:30<6:56:23,  2.20s/it] Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


...

{'eval_loss': 1.8354640007019043, 'eval_runtime': 1068.1852, 'eval_samples_per_second': 8.016, 'eval_steps_per_second': 4.009, 'epoch': 1.0}


 35%|███▌      | 4500/12846 [3:11:13<5:20:41,  2.31s/it]   Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


{'loss': 1.2649, 'grad_norm': 114.33692932128906, 'learning_rate': 1.2993928070994864e-05, 'epoch': 1.05}


 37%|███▋      | 4735/12846 [3:20:56<5:25:23,  2.41s/it] 

KeyboardInterrupt: 

In [12]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

100%|██████████| 162/162 [00:38<00:00,  4.16it/s]

{'eval_loss': 1.839111089706421, 'eval_runtime': 39.1966, 'eval_samples_per_second': 8.266, 'eval_steps_per_second': 4.133, 'epoch': 3.0}


In [41]:
# Save the fine-tuned model
model.save_pretrained('./fine-tuned-bart')
tokenizer.save_pretrained('./fine-tuned-bart')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('./fine-tuned-bart\\tokenizer_config.json',
 './fine-tuned-bart\\special_tokens_map.json',
 './fine-tuned-bart\\vocab.json',
 './fine-tuned-bart\\merges.txt',
 './fine-tuned-bart\\added_tokens.json')

In [43]:
# Load the fine-tuned model and tokenizer
ft_model = BartForConditionalGeneration.from_pretrained('./fine-tuned-bart')
tokenizer = BartTokenizer.from_pretrained('./fine-tuned-bart')

### Test generating summary of the task name

In [60]:
def summarize_phrase(ft_model, phrase, min_len=4, max_len=10):
    # Tokenize the input phrase
    inputs = tokenizer(phrase, max_length=150, truncation=True, return_tensors="pt")

    summary_ids = ft_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=max_len,
        min_length=min_len,
        num_beams=4,
        length_penalty=2.0,
        early_stopping=True
    )

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [61]:
input_phrase = "Assemble the system into a prototype vehicle that we can verify if the brake system can work properly via some specific assessments carried out by another team"

In [69]:
summary = summarize_phrase(ft_model, input_phrase, 20, 20)
print(summary)

automicianifiedkekeviclevicleiclevifiedkev


In [32]:
inputs

{'input_ids': tensor([[    0,  1620, 26381,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0]])}

In [35]:
tokenizer.decode(summary_ids[0], skip_special_tokens=True)

''

In [67]:
def summarize_text(text, model, max_length=150, min_length=40, num_beams=4):
    # Tokenize the input text
    inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
    
    # Generate summary
    summary_ids = model.generate(inputs, max_length=max_length, min_length=min_length, num_beams=num_beams, length_penalty=2.0, early_stopping=False)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return summary

In [68]:
summary = summarize_text(docs[0], ft_model, 20, 20)
print(summary)

automifiedvehiclevomicianvomifiedvicianvician
